In [2]:
ls

bert_imdb.ipynb     cnn_kd_twitter.ipynb      lstm_baseline.ipynb
bert_twitter.ipynb  cnn_twitter.ipynb         lstm_basline_Twitter.ipynb
cnn_imdb.ipynb      kd_atten_lstm_imdb.ipynb  lstm_kd_atten_twitter.ipynb
cnn_kd.ipynb        kd_lstm_imdb.ipynb        lstm_kd_twitter.ipynb


In [3]:
cd ..

/lustre03/project/6001103/dongxx


In [4]:
ls

bert-base-uncased/  read_data.ipynb     slurm-27677361.out  slurm-27769053.out
ca.sh               README.md           slurm-27678712.out  slurm-27789975.out
data140/            result/             slurm-27680260.out  slurm-27792112.out
dataset.ipynb       slurm-27063490.out  slurm-27684303.out  slurm-27792911.out
ENV/                slurm-27238879.out  slurm-27702914.out  slurm-27795501.out
glove.6B.100d.txt   slurm-27240532.out  slurm-27702974.out  slurm-27798306.out
glove.6B.200d.txt   slurm-27286357.out  slurm-27716640.out  slurm-27825402.out
glove.6B.300d.txt   slurm-27292119.out  slurm-27733599.out  slurm-27833275.out
glove.6B.50d.txt    slurm-27299984.out  slurm-27736298.out  slurm-27837414.out
IMDB_data/          slurm-27312560.out  slurm-27739069.out  t.sh
LSTMbaseline/       slurm-27312564.out  slurm-27741364.out  twitter_baseline/
lstm.sh             slurm-27331268.out  slurm-27761768.out  utils.ipynb
mimiciii/           slurm-27337463.out  slurm-27764643.out  w.sh
mini3.ipy

In [5]:
cd LSTMbaseline/

/lustre03/project/6001103/dongxx/LSTMbaseline


In [6]:
from models import CNN_Baseline,LSTMBaseline
from run_lstm_baseline_classifier import validate, prepare_dateset,generate_batch,weight_matrix,epoch_time
import torch
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader
from torchtext.vocab import GloVe,Vocab,Vectors
from tqdm import tqdm
from utils import IMDB_indexing, pad_sequenc,pad_sequencing
from models import CNN_Baseline,LSTMBaseline
import torchtext.vocab
import csv
import pandas as pd
import argparse
import logging
import os
import pickle
import sys
import config
config.seed_torch()
from collections import Counter
import time
import copy

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')



In [8]:
test = '/home/dongxx/projects/def-mercer/dongxx/IMDB_data/test.csv'
valid = '/home/dongxx/projects/def-mercer/dongxx/IMDB_data/valid.csv'
train ='/home/dongxx/projects/def-mercer/dongxx/IMDB_data/train.csv'

In [9]:
counter2 = Counter({'<unk>': 400002, '<pad>': 400001})
glove = Vectors(name='../glove.6B.100d.txt')
f = open('../glove.6B.{}d.txt'.format(100), 'r')
loop = tqdm(f)
vob = {}
loop.set_description('Load Glove')
for i,line in enumerate(loop):
    values = line.split()
    word = values[0]
    vob[word] =400000-i
counter1 = copy.deepcopy(vob)
f.close()
counter1.update(counter2)
vocab = Vocab(counter1)

Load Glove: : 400000it [00:02, 172622.38it/s]


In [10]:
train_dataset, validation_dataset,test_dataset,labellist = prepare_dateset(train, valid,test,vocab)
testing = DataLoader(test_dataset, collate_fn= generate_batch, batch_size=32, shuffle=False)
LSTM_model =LSTMBaseline(vocab_size = 400002,hidden_dim =256 , n_layers = 1, dropout = 0.25, number_class = 2, bidirectional = False, embedding_dim =100)
LSTM_model.embedding_layer.weight.data.copy_(weight_matrix(vocab,glove)).to(device)
LSTM_model.embedding_layer.weight.data[1] = torch.zeros(100)
LSTM_model.embedding_layer.weight.data[0] = torch.zeros(100)
LSTM_model.to(device)
criterion = nn.CrossEntropyLoss()
criterion.to(device)
LSTM_model.load_state_dict(torch.load('/home/dongxx/projects/def-mercer/dongxx/Model_parameter/kd_lstm.pt'))

Start loading training data
Finish loading training data
Start loading validation data
Finish loading validation data
Start loading testing data
Finish loading testing data
prepare training and test sets


20000lines [00:03, 5588.76lines/s]
5000lines [00:00, 5802.21lines/s]
25000lines [00:04, 6049.53lines/s]


building vocab


/home/dongxx/jupyter_py3/lib/python3.7/site-packages/torch/nn/modules/rnn.py:60: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.25 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


<All keys matched successfully>

In [11]:
start_time = time.time()
test_loss, test_acc,flat_list = validate(testing,LSTM_model,criterion,device)
end_time = time.time()
epoch_mins, epoch_secs = epoch_time(start_time, end_time)
print(f' | Epoch Time: {epoch_mins}m {epoch_secs}s')



 | Epoch Time: 0m 11s


In [12]:
print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc * 100:.2f}%')

Test Loss: 0.282 | Test Acc: 88.75%


In [13]:
pred_list =[item.cpu().numpy() for sublist in flat_list for item in sublist]

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import matplotlib.pyplot as plt
import itertools

In [15]:
def plot_confusion_matrix(cm, classes,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """

    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title, fontsize=30)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90, fontsize=22)
    plt.yticks(tick_marks, classes, fontsize=22)

    fmt = '.2f'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label', fontsize=25)
    plt.xlabel('Predicted label', fontsize=25)

In [ ]:
cnf_matrix = confusion_matrix(labellist, pred_list)
plt.figure(figsize=(12,12))
plot_confusion_matrix(cnf_matrix, classes=[0,1], title="Confusion matrix")
plt.show()

In [ ]:
print(classification_report(labellist, pred_list))